In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext 
spark= SparkSession.builder.appName("test").getOrCreate()

In [2]:
df=spark.read.format("csv").load("C:/Users/Shivaraj/pyspark work/supermarket_sales1.csv",header=True,inferSchema=True)

In [3]:
df.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)



In [4]:
df.show(10,False)

+-----------+---------+------+----------------------+----------+--------+-------+--------+----------+
|Invoice ID |City     |Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date      |
+-----------+---------+------+----------------------+----------+--------+-------+--------+----------+
|750-67-8428|Yangon   |null  |Health and beauty     |74.69     |null    |26.1415|548.9715|01-05-2019|
|226-31-3081|Naypyitaw|Female|Electronic accessories|15.28     |5       |3.82   |80.22   |03-08-2019|
|631-41-3108|Yangon   |Male  |Home and lifestyle    |46.33     |7       |16.2155|340.5255|03-03-2019|
|123-19-1176|Yangon   |Male  |Health and beauty     |58.22     |8       |23.288 |489.048 |1/27/2019 |
|373-73-7910|null     |Male  |null                  |86.31     |7       |30.2085|634.3785|02-08-2019|
|699-14-3026|null     |Male  |null                  |85.39     |7       |29.8865|null    |3/25/2019 |
|355-53-5943|Yangon   |Female|null                  |68.84     |6       |20.652 |4

In [5]:
df2= df.filter(df['Total'] >= 500)

In [6]:
df2= df.filter(
    (df['Product line'] == 'Health and beauty') & 
    (df['Total']>=500))

df2.show()

+-----------+---------+------+-----------------+----------+--------+-------+--------+----------+
| Invoice ID|     City|Gender|     Product line|Unit price|Quantity| Tax 5%|   Total|      Date|
+-----------+---------+------+-----------------+----------+--------+-------+--------+----------+
|750-67-8428|   Yangon|  null|Health and beauty|     74.69|    null|26.1415|548.9715|01-05-2019|
|829-34-3910|   Yangon|Female|Health and beauty|     71.38|      10|  35.69|  749.49| 3/29/2019|
|656-95-9349|   Yangon|Female|Health and beauty|     68.93|       7|24.1255|506.6355|03-11-2019|
|370-41-7321| Mandalay|  Male|Health and beauty|     56.69|       9|25.5105|535.7205| 2/27/2019|
|635-40-6220|   Yangon|  Male|Health and beauty|      89.6|       8|  35.84|  752.64|02-07-2019|
|870-72-4431|Naypyitaw|Female|Health and beauty|     99.19|       6| 29.757| 624.897| 1/21/2019|
|152-08-9985| Mandalay|  Male|Health and beauty|     64.36|       9| 28.962| 608.202|03-12-2019|
|766-85-7061| Mandalay|  Male|

In [7]:
# df.groupBy('Health and beauty').min('Total').show()

df3= df.filter(
    (df['Product line'] == 'Health and beauty'))
df3.show()

+-----------+---------+------+-----------------+----------+--------+-------+--------+----------+
| Invoice ID|     City|Gender|     Product line|Unit price|Quantity| Tax 5%|   Total|      Date|
+-----------+---------+------+-----------------+----------+--------+-------+--------+----------+
|750-67-8428|   Yangon|  null|Health and beauty|     74.69|    null|26.1415|548.9715|01-05-2019|
|123-19-1176|   Yangon|  Male|Health and beauty|     58.22|       8| 23.288| 489.048| 1/27/2019|
|665-32-9167|   Yangon|Female|Health and beauty|     36.26|    null|  3.626|  76.146|01-10-2019|
|829-34-3910|   Yangon|Female|Health and beauty|     71.38|      10|  35.69|  749.49| 3/29/2019|
|656-95-9349|   Yangon|Female|Health and beauty|     68.93|       7|24.1255|506.6355|03-11-2019|
|371-85-5789| Mandalay|  Male|Health and beauty|     87.98|       3| 13.197| 277.137|03-05-2019|
|848-62-7243|   Yangon|  Male|Health and beauty|     24.89|       9|11.2005|235.2105| 3/15/2019|
|595-11-5460|   Yangon|  Male|

In [8]:
df3.count()

152

In [9]:
(df3.groupBy('Total').avg()).show()

+--------+---------------+-------------+-----------+----------+
|   Total|avg(Unit price)|avg(Quantity)|avg(Tax 5%)|avg(Total)|
+--------+---------------+-------------+-----------+----------+
|  76.146|          36.26|         null|      3.626|    76.146|
| 697.935|          66.47|         10.0|     33.235|   697.935|
|  545.37|          51.94|         10.0|      25.97|    545.37|
|  75.474|          17.97|          4.0|      3.594|    75.474|
| 633.108|          75.37|          8.0|     30.148|   633.108|
| 628.173|          99.71|          6.0|     29.913|   628.173|
| 166.635|          15.87|         10.0|      7.935|   166.635|
| 145.971|          69.51|          2.0|      6.951|   145.971|
|    90.3|          10.75|          8.0|        4.3|      90.3|
| 91.4025|          17.41|          5.0|     4.3525|   91.4025|
|548.9715|          74.69|         null|    26.1415|  548.9715|
|223.0725|          30.35|          7.0|    10.6225|  223.0725|
|110.0925|          20.97|          5.0|

In [10]:
from pyspark.sql.functions import *

df4=df.groupBy('Product line').avg('Total').select('Product line',col("avg(Total)").alias("average"))
df4.show()

+--------------------+------------------+
|        Product line|           average|
+--------------------+------------------+
|  Home and lifestyle|      336.63695625|
| Fashion accessories| 306.4693728813559|
|                null|325.40812500000004|
|   Health and beauty|323.64301973684223|
|Electronic access...| 317.5433263473054|
|  Food and beverages|323.53813872832376|
|   Sports and travel| 330.2330181818182|
+--------------------+------------------+



In [11]:
df4.sort(col('average').desc()).show()


+--------------------+------------------+
|        Product line|           average|
+--------------------+------------------+
|  Home and lifestyle|      336.63695625|
|   Sports and travel| 330.2330181818182|
|                null|325.40812500000004|
|   Health and beauty|323.64301973684223|
|  Food and beverages|323.53813872832376|
|Electronic access...| 317.5433263473054|
| Fashion accessories| 306.4693728813559|
+--------------------+------------------+



In [12]:
df2.show()

+-----------+---------+------+-----------------+----------+--------+-------+--------+----------+
| Invoice ID|     City|Gender|     Product line|Unit price|Quantity| Tax 5%|   Total|      Date|
+-----------+---------+------+-----------------+----------+--------+-------+--------+----------+
|750-67-8428|   Yangon|  null|Health and beauty|     74.69|    null|26.1415|548.9715|01-05-2019|
|829-34-3910|   Yangon|Female|Health and beauty|     71.38|      10|  35.69|  749.49| 3/29/2019|
|656-95-9349|   Yangon|Female|Health and beauty|     68.93|       7|24.1255|506.6355|03-11-2019|
|370-41-7321| Mandalay|  Male|Health and beauty|     56.69|       9|25.5105|535.7205| 2/27/2019|
|635-40-6220|   Yangon|  Male|Health and beauty|      89.6|       8|  35.84|  752.64|02-07-2019|
|870-72-4431|Naypyitaw|Female|Health and beauty|     99.19|       6| 29.757| 624.897| 1/21/2019|
|152-08-9985| Mandalay|  Male|Health and beauty|     64.36|       9| 28.962| 608.202|03-12-2019|
|766-85-7061| Mandalay|  Male|

In [13]:
df.rdd.getNumPartitions()

1

In [14]:
type(df2)

pyspark.sql.dataframe.DataFrame

In [15]:
df.write.format("json").save('C:/Users/shivaraj/outputnew3')